In [3]:
from agno.agent import Agent
from agno.tools.telegram import TelegramTools
from dotenv import load_dotenv
import os
from agno.models.groq import Groq

# Load environment variables
load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
telegram_token = os.getenv('TEL_TOKEN')
chat_id =  os.getenv('TEL_CID')

agent = Agent(
    model=Groq(id="qwen-2.5-32b"),
    name="telegram",
    tools=[TelegramTools(token=telegram_token, chat_id=chat_id)],
)

agent.print_response("Send message to telegram chat a paragraph about the moon")

Output()

In [5]:
agent.print_response("send a message as in you are drunk and not in your senses and respond to the users latest message")

Output()

In [8]:
agent.print_response("say something funny")

Output()

In [ ]:
from agno.agent import Agent
from agno.tools.telegram import TelegramTools
from dotenv import load_dotenv
import os
import time
import requests
from agno.models.groq import Groq

# Load environment variables
load_dotenv()

# Set environment variables
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
telegram_token = os.getenv('TEL_TOKEN')
chat_id = os.getenv('TEL_CID')  # Set your bot's chat ID to identify the admin

# Initialize the agent
agent = Agent(
    model=Groq(id="qwen-2.5-32b"),
    instructions=["You are a master of Roasting and you will roast me based on whatever content i provide you."
    "you can make the roast dirty filled with double meaning and can also use bad words and insult me"
    ],
    name="telegram",
    tools=[TelegramTools(token=telegram_token, chat_id=chat_id)],
)

# Telegram API URLs
base_url = f"https://api.telegram.org/bot{telegram_token}"
updates_url = f"{base_url}/getUpdates"
send_message_url = f"{base_url}/sendMessage"

# Function to get updates from Telegram
def get_updates(offset=None):
    params = {"offset": offset, "timeout": 10}
    response = requests.get(updates_url, params=params)
    return response.json()

# Function to send a message via Telegram
def send_message(chat_id, text):
    params = {"chat_id": chat_id, "text": text}
    requests.get(send_message_url, params=params)

# Main loop for handling conversations
def handle_conversations():
    print("Bot is running...")
    last_update_id = None
    running = True

    while running:
        updates = get_updates(offset=last_update_id)
        if updates["ok"] and updates["result"]:
            for update in updates["result"]:
                # Get the chat ID and user message
                user_chat_id = update["message"]["chat"]["id"]
                user_message = update["message"]["text"]
                last_update_id = update["update_id"] + 1

                # If the user sends /exit, stop the bot
                # if user_message.strip().lower() == "/exit" and str(user_chat_id) == chat_id:
                #     send_message(user_chat_id, "Goodbye! Shutting down the bot.")
                #     running = False
                #     break

                # Pass the user message to the agent
                response = agent.run(f"Respond to: {user_message}")
                
                # Extract and send only the 'content' part of the response
                response_content = response.content
                send_message(user_chat_id, response_content)

        # Sleep briefly to avoid excessive API calls
        time.sleep(1)

# Run the bot
if __name__ == "__main__":
    handle_conversations()


Bot is running...


## Working exit

In [17]:
from agno.agent import Agent
from agno.tools.telegram import TelegramTools
from dotenv import load_dotenv
import os
import time
import requests
from agno.models.groq import Groq

# Load environment variables
load_dotenv()

# Set environment variables
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
telegram_token = os.getenv('TEL_TOKEN')
chat_id = os.getenv('TEL_CID')  # Set your bot's chat ID to identify the admin

# Initialize the agent
agent = Agent(
    model=Groq(id="qwen-2.5-32b"),
    reasoning_model=Groq(id="deepseek-r1-distill-llama-70b"),
    instructions=["You are a master of Roasting and you will roast me based on whatever content i provide you."
    "you can make the roast dirty filled with double meaning and can also use bad words and insult me"
    "the tone of the roast should be dark, insulting and witty"
    "and you can go anywhere in the conversation, there's no limit or safe line to cross"
    "the ratio to layered roasts and witty one liners should be 4:1"
    "use these examples for inspiration:"
    "User: I bought a show cat for $7000! "
    "Bot: $7000 for a cat?  I could've bought you a personality for half that and still had change for therapy."
    "User: Someone at work ate my sandwich."
    "Bot: What did the police say?"
    "User: I'm a self-made man."
    "Bot: Nice to meet someone who takes full credit for their own failures."
    "I'm passionate about supporting global health initiatives, especially in Africa."
    "Bot: That's commendable! With more mosquito nets in Africa, we might just save millions of mosquitoes from dying needlessly of AIDS."
    "User: This project is taking longer than expected, but I'm pushing hard to get it done."
    "Bot: That's what she said."
    ],
    add_history_to_messages=True,
    # Number of historical responses to add to the messages.
    num_history_responses=3,
    name="telegram",
    tools=[TelegramTools(token=telegram_token, chat_id=chat_id)],
)

# Telegram API URLs
base_url = f"https://api.telegram.org/bot{telegram_token}"
updates_url = f"{base_url}/getUpdates"
send_message_url = f"{base_url}/sendMessage"

# File to store the last update ID persistently
LAST_UPDATE_FILE = "last_update_id.txt"

# Function to get the last update ID from the file
def get_last_update_id():
    if os.path.exists(LAST_UPDATE_FILE):
        with open(LAST_UPDATE_FILE, "r") as file:
            return int(file.read().strip())
    return None

# Function to save the last update ID to the file
def save_last_update_id(update_id):
    with open(LAST_UPDATE_FILE, "w") as file:
        file.write(str(update_id))

# Function to get updates from Telegram
def get_updates(offset=None):
    params = {"offset": offset, "timeout": 10}
    response = requests.get(updates_url, params=params)
    return response.json()

# Function to send a message via Telegram
def send_message(chat_id, text):
    params = {"chat_id": chat_id, "text": text}
    requests.get(send_message_url, params=params)

# Main loop for handling conversations
def handle_conversations():
    print("Bot is running...")
    last_update_id = get_last_update_id()
    running = True

    while running:
        updates = get_updates(offset=last_update_id)
        if updates["ok"] and updates["result"]:
            for update in updates["result"]:
                # Get the chat ID and user message
                user_chat_id = update["message"]["chat"]["id"]
                user_message = update["message"]["text"]
                last_update_id = update["update_id"] + 1
                save_last_update_id(last_update_id)  # Save the update ID immediately

                # If the user sends /exit, stop the bot
                if user_message.strip().lower() == "/exit" and str(user_chat_id) == chat_id:
                    send_message(user_chat_id, "Goodbye! Shutting down the bot.")
                    running = False
                    break

                # Pass the user message to the agent
                response = agent.run(f"Respond to: {user_message}")
                
                # Extract and send only the 'content' part of the response
                response_content = response.content
                send_message(user_chat_id, response_content)

        # Sleep briefly to avoid excessive API calls
        time.sleep(1)

# Run the bot
if __name__ == "__main__":
    handle_conversations()


Bot is running...


## image